In [ ]:
# mount drive first
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Link to drive with images for reference
# https://drive.google.com/drive/folders/1WIZB_ZLItOE0XkjQg2FsLZIuoSnIcYQp
!apt-get install -y p7zip-full
DRIVE_ZIP_PATH_1 = '/content/drive/MyDrive/Colab Notebooks/RevealAI/48K First Sample/test.7z'
DRIVE_ZIP_PATH_2 = '/content/drive/MyDrive/Colab Notebooks/RevealAI/48K First Sample/train.7z'
DRIVE_ZIP_PATH_3 = '/content/drive/MyDrive/Colab Notebooks/RevealAI/48K First Sample/validation.7z'


# # Local temporary disk destination
LOCAL_DESTINATION = '/content/'

# # Execute the copy command

!cp "{DRIVE_ZIP_PATH_1}" "{LOCAL_DESTINATION}"
!cp "{DRIVE_ZIP_PATH_2}" "{LOCAL_DESTINATION}"
!cp "{DRIVE_ZIP_PATH_3}" "{LOCAL_DESTINATION}"

print(f" Copied data to local Colab disk.")
# # Path to the ZIP file on the local disk

LOCAL_ZIP_PATH_1 = '/content/test.7z'
LOCAL_ZIP_PATH_2 = '/content/train.7z'
LOCAL_ZIP_PATH_3 = '/content/validation.7z'

# # Execute the unzip command
# # -q: quiet (less terminal output)
# # -d /content/: extract contents to the /content/ directory

!7z x "{LOCAL_ZIP_PATH_1}" -o/content/
!7z x "{LOCAL_ZIP_PATH_2}" -o/content/
!7z x "{LOCAL_ZIP_PATH_3}" -o/content/
print(" Unzipping complete! Your data is now fast to access.")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
 Copied data to local Colab disk.

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,12 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (50657),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 714238854 bytes (682 MiB)

Extracting archive: /content/test.7z
--
Path = /content/test.7z
Type = 7z
Physical Size = 714238854
Headers Size = 103910
Method = LZMA:23
Solid = +
Blocks = 1

  0%      0% - test/ai/000_biggan_00130.jpg                                     1% 50 - test/ai/054_biggan_00066.jpg

In [ ]:
!pip install timm
!pip install transformers timm ftfy

In [ ]:
# u have to get ride of the empty files or the model wont run
import os

folders = [
    "/content/train/ai",
    "/content/train/real",
    "/content/test/ai",
    "/content/test/real",
    "/content/validation/ai",
    "/content/validation/real"
]

count_removed = 0
for folder in folders:
  for f in os.listdir(folder):
    path = os.path.join(folder, f)
    if os.path.isfile(path) and f.lower().endswith(('.jpg', '.jpeg')):
        if os.path.getsize(path) == 0:
            os.remove(path)
            count_removed += 1
            # print(f"Deleted empty file: {path}")
print('Count of deleted files : ', count_removed)


In [ ]:
# imports
import os, sys,torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
from PIL import Image
import torch
import torch.nn.functional as F
import timm
import torch.nn as nn
import matplotlib.pyplot as plt
import random
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torchvision.models import shufflenet_v2_x1_0, ShuffleNet_V2_X1_0_Weights

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_path = "/content/train"
val_path  = "/content/validation"
test_path = "/content/test"

train_data = datasets.ImageFolder(train_path, transform=transform)
val_data  = datasets.ImageFolder(val_path,  transform=transform)
test_data = datasets.ImageFolder(test_path,  transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader  = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data,  batch_size=32, shuffle=False)

class_names = train_data.classes
num_classes = len(class_names)
num_classes

In [ ]:
def test_model(model_path, model_constructor, model_name="model"):


    model = model_constructor()
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()

    preds = []
    trues = []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            preds.extend(predicted.cpu().numpy())
            trues.extend(labels.cpu().numpy())

    acc  = accuracy_score(trues, preds)
    prec = precision_score(trues, preds, average="weighted", zero_division=0)
    rec  = recall_score(trues, preds, average="weighted", zero_division=0)
    f1   = f1_score(trues, preds,     average="weighted", zero_division=0)

    print(f"Test Accuracy : {acc:.4f}")
    print(f"Test Precision: {prec:.4f}")
    print(f"Test Recall   : {rec:.4f}")
    print(f"Test F1 Score : {f1:.4f}")

    return acc, prec, rec, f1

In [ ]:
test_model(
    model_path="/content/drive/MyDrive/resnet50.pth",
    model_constructor=lambda: timm.create_model("resnet50", pretrained=False, num_classes=2),
    model_name="ResNet50"
)

In [ ]:
print('Standard photos model')
test_model(
    model_path="/content/drive/MyDrive/shufflenetv2.pth",
    model_constructor=lambda: shufflenet_v2_x1_0(weights=None, num_classes=2),
    model_name="ShuffleNetV2"
)

In [ ]:
def train(model, criterion, optimizer, train_loader, val_loader, epochs=10, model_name="model"):
    metrics = {
        "train_loss": [],
        "val_loss": [],
        "train_acc": [],
        "val_acc": []
    }

    model.to(device)
    best_val_acc = 0
    for epoch in range(epochs):
        model.train()
        running_loss = 0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_train_loss = running_loss / len(train_loader.dataset)
        epoch_train_acc = correct / total

        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * images.size(0)

                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        epoch_val_loss = val_loss / len(val_loader.dataset)
        epoch_val_acc = val_correct / val_total

        metrics["train_loss"].append(epoch_train_loss)
        metrics["train_acc"].append(epoch_train_acc)
        metrics["val_loss"].append(epoch_val_loss)
        metrics["val_acc"].append(epoch_val_acc)

        print(
            f"Epoch {epoch+1}/{epochs} "
            f"Train Loss: {epoch_train_loss:.4f}  Train Acc: {epoch_train_acc:.4f} | "
            f"Val Loss: {epoch_val_loss:.4f}  Val Acc: {epoch_val_acc:.4f}"
        )

        if epoch_val_acc > best_val_acc:
            best_val_acc = epoch_val_acc
            torch.save(model.state_dict(), f"{model_name}_cropped.pth")


    plot(model_name, metrics, epochs)

    return model

In [ ]:
def plot(model_name, metrics, epochs):

    x = range(1, epochs + 1)
    plt.figure(figsize=(8, 5))
    plt.plot(x, metrics["train_loss"], label="Train Loss")
    plt.plot(x, metrics["val_loss"],   label="Val Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title(f"{model_name} Loss Curve")
    plt.legend()
    plt.savefig(f"{model_name}_loss.png")
    plt.close()


    plt.figure(figsize=(8, 5))
    plt.plot(x, metrics["train_acc"], label="Train Accuracy")
    plt.plot(x, metrics["val_acc"],   label="Val Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title(f"{model_name} Accuracy Curve")
    plt.legend()
    plt.savefig(f"{model_name}_accuracy.png")
    plt.close()

In [ ]:
def evaluate(model, model_name="model"):
    model.eval()
    preds = []
    trues = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            preds.extend(predicted.cpu().numpy())
            trues.extend(labels.cpu().numpy())

    acc = accuracy_score(trues, preds)
    prec = precision_score(trues, preds, average="weighted", zero_division=0)
    rec = recall_score(trues, preds, average="weighted", zero_division=0)
    f1 = f1_score(trues, preds, average="weighted", zero_division=0)


    print("Accuracy :", acc)
    print("Precision:", prec)
    print("Recall   :", rec)
    print("F1 Score :", f1)

    return acc, prec, rec, f1


In [ ]:

# Link to drive with images for reference
# https://drive.google.com/drive/folders/1WIZB_ZLItOE0XkjQg2FsLZIuoSnIcYQp
!apt-get install -y p7zip-full
DRIVE_ZIP_PATH_1 = '/content/drive/MyDrive/Colab Notebooks/RevealAI/48K Scaled/test.7z'
DRIVE_ZIP_PATH_2 = '/content/drive/MyDrive/Colab Notebooks/RevealAI/48K Scaled/train.7z'
DRIVE_ZIP_PATH_3 = '/content/drive/MyDrive/Colab Notebooks/RevealAI/48K Scaled/validation.7z'


# # Local temporary disk destination
LOCAL_DESTINATION = '/content/'

# # Execute the copy command

!cp "{DRIVE_ZIP_PATH_1}" "{LOCAL_DESTINATION}"
!cp "{DRIVE_ZIP_PATH_2}" "{LOCAL_DESTINATION}"
!cp "{DRIVE_ZIP_PATH_3}" "{LOCAL_DESTINATION}"

print(f" Copied data to local Colab disk.")
# # Path to the ZIP file on the local disk

LOCAL_ZIP_PATH_1 = '/content/test.7z'
LOCAL_ZIP_PATH_2 = '/content/train.7z'
LOCAL_ZIP_PATH_3 = '/content/validation.7z'

# # Execute the unzip command
# # -q: quiet (less terminal output)
# # -d /content/: extract contents to the /content/ directory

!7z x "{LOCAL_ZIP_PATH_1}" -o/content/
!7z x "{LOCAL_ZIP_PATH_2}" -o/content/
!7z x "{LOCAL_ZIP_PATH_3}" -o/content/
print(" Unzipping complete! Your data is now fast to access.")

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_path = "/content/DL_Temp/train"
val_path  = "/content/DL_Temp/validation"
test_path = "/content/DL_Temp/test"

train_data = datasets.ImageFolder(train_path, transform=transform)
val_data  = datasets.ImageFolder(val_path,  transform=transform)
test_data = datasets.ImageFolder(test_path,  transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader  = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data,  batch_size=32, shuffle=False)

class_names = train_data.classes
num_classes = len(class_names)
num_classes

In [ ]:
print('Scaled Model')
torch.cuda.empty_cache()
model = timm.create_model("resnet50", pretrained=True, num_classes=2)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

resnet_model = train(model, criterion, optimizer, train_loader, val_loader, epochs=10,model_name="resnet50")
evaluate(resnet_model, model_name="ResNet50")

In [ ]:
print('Scaled Model')
test_model(
    model_path="/content/resnet50.pth",
    model_constructor=lambda: timm.create_model("resnet50", pretrained=False, num_classes=2),
    model_name="ResNet50"
)

In [ ]:
print('Scaled Model')
test_model(
    model_path="/content/drive/MyDrive/Shufflenet_model_best_scaled.pth",
    model_constructor=lambda: shufflenet_v2_x1_0(weights=None, num_classes=2),
    model_name="ShuffleNetV2"
)

In [ ]:

# Link to drive with images for reference
# https://drive.google.com/drive/folders/1WIZB_ZLItOE0XkjQg2FsLZIuoSnIcYQp
!apt-get install -y p7zip-full
DRIVE_ZIP_PATH_1 = '/content/drive/MyDrive/Colab Notebooks/RevealAI/48K Cropped/test.7z'
DRIVE_ZIP_PATH_2 = '/content/drive/MyDrive/Colab Notebooks/RevealAI/48K Cropped/train.7z'
DRIVE_ZIP_PATH_3 = '/content/drive/MyDrive/Colab Notebooks/RevealAI/48K Cropped/validation.7z'


# # Local temporary disk destination
LOCAL_DESTINATION = '/content/'

# # Execute the copy command

!cp "{DRIVE_ZIP_PATH_1}" "{LOCAL_DESTINATION}"
!cp "{DRIVE_ZIP_PATH_2}" "{LOCAL_DESTINATION}"
!cp "{DRIVE_ZIP_PATH_3}" "{LOCAL_DESTINATION}"

print(f" Copied data to local Colab disk.")
# # Path to the ZIP file on the local disk

LOCAL_ZIP_PATH_1 = '/content/test.7z'
LOCAL_ZIP_PATH_2 = '/content/train.7z'
LOCAL_ZIP_PATH_3 = '/content/validation.7z'

# # Execute the unzip command
# # -q: quiet (less terminal output)
# # -d /content/: extract contents to the /content/ directory

!7z x "{LOCAL_ZIP_PATH_1}" -o/content/
!7z x "{LOCAL_ZIP_PATH_2}" -o/content/
!7z x "{LOCAL_ZIP_PATH_3}" -o/content/
print(" Unzipping complete! Your data is now fast to access.")

In [ ]:
print('Cropped Model')
torch.cuda.empty_cache()
model = timm.create_model("resnet50", pretrained=True, num_classes=2)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

resnet_model = train(model, criterion, optimizer, train_loader, val_loader, epochs=10,model_name="resnet50")
evaluate(resnet_model, model_name="ResNet50")

In [ ]:
print('Cropped Model')
test_model(
    model_path="/content/drive/MyDrive/Shufflenet_model_best.pth",
    model_constructor=lambda: shufflenet_v2_x1_0(weights=None, num_classes=2),
    model_name="ShuffleNetV2"
)